In [ ]:
import sys
sys.path.append('C:\\Users\\Shih-Yi\\Documents\\GitHub\\swac\\+swac_glm')

import os
import datajoint as dj
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# create virtual modules for swac and swac_beh
swac = dj.create_virtual_module('swac','swac')
swac_beh = dj.create_virtual_module('swac_beh','swac_beh')
swac_glm = dj.create_virtual_module('swac_glm','swac_glm')

from pynwb import NWBHDF5IO, NWBFile, TimeSeries, load_namespaces, get_class
from pynwb.file import Subject
from pynwb.behavior import SpatialSeries, Position, BehavioralTimeSeries
from pynwb.ophys import ImageSegmentation, OpticalChannel, ImagingPlane, RoiResponseSeries, DfOverF
from pynwb.base import Images
from pynwb.image import GrayscaleImage, ImageSeries

from ndx_harvey_swac import LabMetaDataSession

from datetime import datetime, timedelta
from dateutil import tz
import h5py
from numpy.matlib import repmat

In [ ]:
sugery_note = {
    3:'cranial window creation date:2017-04-03, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-04-03; GCaMP6s injection date:2017-04-03; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    4:'cranial window creation date:2017-05-03, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-03; GCaMP6s injection date:2017-05-03; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    5:'cranial window creation date:2017-05-29, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-29; GCaMP6s injection date:2017-05-29; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    6:'cranial window creation date:2017-06-05, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-06-05; GCaMP6s injection date:2017-06-12; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    7:'cranial window creation date:2017-05-12, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-12; GCaMP6s injection date:2017-07-05; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    8:'cranial window creation date:2017-05-17, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-17; GCaMP6s injection date:2017-07-12; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    9:'cranial window creation date:2017-05-25, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-25; GCaMP6s injection date:2017-07-26; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    10:'cranial window creation date:2017-05-26, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-26; GCaMP6s injection date:2017-09-11; performed by Shih-Yi Tseng and Selmaan N. Chettih'
}

virus_note = {
    3:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x3 sites, PM x1, AM x0, MM x1, RSC x1, visA x1, 1/10 dilution, 140nl per site in L23 and 140nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.95, L 0.6, D 1.0), (A 0.95, L 0.6, D 0.3), (A 0.95, L 0.8, D 0.3); AAV2retro-Syn-mScarlet in left striatum x3, 1/20 dilution, 300 nl per site, coordinate (mm from bregma): (A 0.95, L 1.2, D 2.1), (A 1.0, L 1.75, D 2.1), (P 0.15, L 2.15, D 2.1); mScarlet expression did not come up when examining it under the 2P microscope (due to the higher dilution)''',
    4:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x3 sites, PM x1, AM x0, MM x1, RSC x0, visA x2, 1/10 dilution, 100nl per site in L23 and 140nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.9, L 0.58, D 0.3), (A 0.9, L 0.58, D 1.0), (A 0.9, L 0.8, D 0.4); AAV2retro-Syn-mScarlet in left striatum x3, 1/5 dilution, 300 nl per site for the anterior sites and 600 nl for the posterior site, coordinate (mm from bregma): (A 1.0, L 1.2, D 2.1), (A 1.0, L 1.5, D 2.1), (P 0.2, L 1.75, D 2.1)''',
    5:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x3 sites, PM x1, AM x1, MM x1, RSC x0, visA x2, 1/5 dilution, 100nl per site in L23 and 140nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.96, L 0.4, D 0.3), (A 0.96, L 0.4, D 1.0), (A 0.96, L 0.8, D 0.4); AAV2retro-Syn-mScarlet in left striatum x3, 1/5 dilution, 300 nl per site for the anterior sites and 600 nl for the posterior site, coordinate (mm from bregma): (A 0.96, L 1.2, D 2.1), (A 0.96, L 1.5, D 2.1), (P 0.24, L 1.8, D 2.1)''',
    6:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x3 sites, PM x1, AM x2, MM x1, RSC x1, visA x2, 1/10 dilution, 100nl per site in L23 and 140nl per site in L5; AAV2retro-Syn-mTagBFP2 in left posterior ACC/M2 x4, undiluted, 300 nl per site, coordinate (mm from bregma): (0, L 0.3, D 0.4), (0, L 0.3, D 0.8), (0, L 0.7, D 0.3), (0, L 0.7, D 0.75); AAV2retro-Syn-mScarlet in left ORBvl x1, ORBl x1, 1/5 dilution, 500 nl per site, coordinate (mm from bregma): (A 2.45, L 0.75, D 1.85), (A 2.45, L 1.26, D 1.85)''',
    7:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x2 sites, PM x1, AM x1, MM x1, RSC x1, visA x2, 1/10 dilution, 70nl per site in L23 and 100nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.7, L 0.5, D 0.3), (A 0.7, L 0.5, D 1.0), (A 1.0, L 0.8, D 0.4); AAV2retro-Syn-mScarlet in left striatum x3, 1/5 dilution, 300 nl per site for the anterior sites and 600 nl for the posterior site, coordinate (mm from bregma): (A 0.8, L 1.2, D 2.1), (A 1.15, L 1.5, D 2.1), (P 0.2, L 1.8, D 2.1)''',
    8:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x2 sites, PM x1, AM x1, MM x1, RSC x1, visA x2, 1/10 dilution, 70nl per site in L23 and 100nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 1.1, L 0.55, D 0.3), (A 1.1, L 0.55, D 1.0), (A 0.97, L 0.8, D 0.4); AAV2retro-Syn-mScarlet in left striatum x3, 1/5 dilution, 300 nl per site for the anterior sites and 600 nl for the posterior site, coordinate (mm from bregma): (A 0.98, L 1.2, D 2.1), (A 0.85, L 1.5, D 2.1), (P 0.152, L 1.8, D 2.1)''',
    9:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x2 sites, PM x1, AM x1, MM x1, RSC x1, visA x2, 1/10 dilution, 70nl per site in L23 and 100nl per site in L5; AAV2retro-Syn-mTagBFP2 in left posterior ACC/M2 x4, undiluted, 300 nl per site, coordinate (mm from bregma): (0, L 0.35, D 0.4), (0, L 0.35, D 0.8), (0, L 0.7, D 0.3), (0, L 0.7, D 0.8); AAV2retro-Syn-mScarlet in left ORBvl x1, ORBl x1, 1/5 dilution, 500 nl per site, coordinate (mm from bregma): (A 2.65, L 0.85, D 1.8), (A 2.6, L 1.35, D 1.75)''',
    10:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x2 sites, PM x1, AM x1, MM x1, RSC x1, visA x2, 1/10 dilution, 70nl per site in L23 and 100nl per site in L5; AAV2retro-Syn-mTagBFP2 in left posterior ACC/M2 x4, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.25, L 0.3, D 0.4), (A 0.25, L 0.3, D 0.85), (A 0.1, L 0.7, D 0.3), (A 0.1, L 0.7, D 0.8); AAV2retro-Syn-mScarlet in left  ORBvl x1, ORBl x1, 1/5 dilution, 500 nl per site, coordinate (mm from bregma): (A 2.45, L 0.75, D 1.8), (A 2.45, L 1.25, D 1.85)'''
}

retro_site = {
    3:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    4:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    5:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    6:{'mTagBFP2':'posterior_ACC_M2','mScarlet':'OFC'},
    7:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    8:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    9:{'mTagBFP2':'posterior_ACC_M2','mScarlet':'OFC'},
    10:{'mTagBFP2':'posterior_ACC_M2','mScarlet':'OFC'}
}

first_session = {
    3:'2017-04-27',
    4:'2017-05-13',
    5:'2017-06-13',
    6:'2017-06-27',
    7:'2017-07-14',
    8:'2017-07-25',
    9:'2017-08-07',
    10:'2017-09-21'
}

In [ ]:
def model_prob_of_action(strat_preds, trial_dict):
    '''Compute model probability of action from taskRNN prediction
    Input:
    strat_preds: policy matrix of taskRNN, in the order of WR, WL, BR, BL
    trial_dict: dictionary containing trial type info
    
    Output:
    full_prob: full model probability of the action
    cho_prob: marginal choice probability
    cue_prob: improvement of action probability given the cue
    '''
    
    # convert cue and choice back to B=1, W=0 & L=1, R=0
    cueB = trial_dict['trialCueB'].reshape(-1,)
    cueB = (cueB - np.min(cueB))/(np.max(cueB) - np.min(cueB))
    
    choL = trial_dict['trialChoL'].reshape(-1,)
    choL = (choL - np.min(choL))/(np.max(choL) - np.min(choL))
    
    # scale policy matrix to be *conditioned* on cue
    cond_pred = strat_preds * 2
    
    # calculate marginal choice probability (order of strat_preds: WR, WL, BR, BL)
    prob_right = (strat_preds[0,:]+strat_preds[2,:]).reshape(1,-1)
    prob_left = (strat_preds[1,:]+strat_preds[3,:]).reshape(1,-1)    
    marg_pred = np.concatenate((prob_right,prob_left), axis = 0)
    
    # get indices for each trials choice
    cond_ind = (cueB*2 + choL).astype(int)
    marg_ind = choL.astype(int)
    
    # calculate
    full_prob = np.array([cond_pred[cond, ind] for ind, cond in enumerate(cond_ind)])
    cho_prob = np.array([marg_pred[marg, ind] for ind, marg in enumerate(marg_ind)])
        
    # break full conditional prediction into marginal and improvement from marginal (cue_prob)
    cue_prob = full_prob - cho_prob
    
    return full_prob, cho_prob, cue_prob


def partition_areas(ml, ap):
    from collections import OrderedDict
    from sklearn.neighbors import NearestNeighbors
    from shapely.geometry import Point, Polygon
    from scipy.io import loadmat
   
    # load area partition
    area = loadmat('C:\\Users\\Shih-Yi\\Documents\\GitHub\\swac\\AllenMouseCCF\\allenAreaPartitions.mat')
    areas = ['V1', 'PM', 'AM', 'MM', 'RSC', 'NP']
    
    # get point objects for sources
    coor = np.stack([ml, ap], axis = 1)
    points = [Point(coor[i,:]) for i in range(coor.shape[0])]
    
    # get polygon objects for area
    all_poly = {a:Polygon(area[a]) for a in areas}
    area_ind = OrderedDict({name:np.array([pt.within(poly) for pt in points]) for name, poly in all_poly.items()})
    
    binary_id = np.zeros((ml.shape[0], len(areas)))
    for i, idx in enumerate(area_ind.values()):
        binary_id[:,i] = idx
    
    all_labels = np.argmax(binary_id, axis = 1)
    bad_ids = np.sum(binary_id, axis = 1) != 1
    if np.sum(bad_ids)>0:
        # for single entry data, use the first True
        if ml.shape[0] == 1:
            prev_true_exists = False
            for idx, (this_area, val) in enumerate(area_ind.items()):
                if not prev_true_exists:
                    if val[0]: 
                        prev_true_exists = True
                elif prev_true_exists:
                    if val[0]:
                        area_ind[this_area] = np.array([False])
        
        # for multiple entry data, find nearest neigbhor
        elif ml.shape[0] >= 1:
            good_ids = ~bad_ids

            good_pts = np.stack((ml[good_ids],ap[good_ids]),axis = 1)
            good_labels = all_labels[good_ids]
            bad_pts = np.stack((ml[bad_ids],ap[bad_ids]),axis = 1)
            NN = NearestNeighbors(n_neighbors=1, radius=0.2).fit(good_pts) 
            neighbors = NN.kneighbors(bad_pts, return_distance=False) 
            new_labels = good_labels[neighbors.flatten()]
            all_labels[bad_ids] = new_labels

            area_ind = OrderedDict({name: all_labels == i for i, name in enumerate(area_ind.keys())})   

    
    area_ind['RSC'] = area_ind.pop('RSC')
    area_ind['A/RL'] = area_ind.pop('NP')
        
    return area_ind

In [ ]:
# fetch all imaging sessions
all_sessions = swac.Session.fetch()

In [ ]:
for i_sess, this_sess in enumerate(all_sessions):
    
    # delete previous nwbfile
    if i_sess > 0:
        del nwbfile
        
        
    #### SESSION INFO PREPARATION ####
    # process key
    key = {'mouse_id': this_sess[0], 'session_date': this_sess[1]}
    mouse_id = key['mouse_id']
    session_date = key['session_date']
    
    print(f'Start processing mouse_id = {mouse_id}, session_date = {session_date}')
    
    # fetch session info
    acq_name, layer, img_type, fov, step_size, laser_power, lz, sync_loc, behav_loc, session_note = (swac.SessionInfo & key).fetch1(
        'acq_name', 'layer', 'type', 'fov', 'step_size', 'laser_power', 'lz','sync_loc', 'behav_loc','notes')
    if session_note=='':
        session_note = None

    if layer == '2/3':
        layer_name = '23'
    else:
        layer_name = 5
        
    if img_type == 'plane':
        img_type_name = 'single_plane'
    elif img_type == 'volume':
        img_type_name = 'multi_plane'
        
    # fetch FOV center location
    center_ml, center_ap, depth = (swac.FOVtoCCF & key).fetch1('center_ml_ccf_mm', 'center_ap_ccf_mm', 'depth_mm')

    # find area that FOV centered on 
    area_ind = partition_areas(np.array(center_ml).reshape(-1,1), np.array(center_ap).reshape(-1,1))
    fov_center_area = [area_name for idx, area_name in enumerate(area_ind.keys()) if area_ind[area_name][0] == True][0]
    if fov_center_area == 'A/RL':
        fov_center_area = 'visA'
        
    # fetch raw behavioral data
    expt, switches, frac_no_checker, penalty, delay, performance, init_block, variables, behav_data = (swac.Behavior & key).fetch1(
     'expt', 'switches', 'frac_no_checker', 'penalty', 'delay', 'performance','init_block', 'variables', 'behav_data')

    if delay is None:
        delay = 0.

    # extract behavioral data
    world = behav_data[0,:]
    lat_vel_verm = behav_data[1,:]
    for_vel_verm = behav_data[2,:]
    lat_pos = behav_data[4,:]
    for_pos = behav_data[5,:]
    trial_phase = behav_data[7,:]
    reward = behav_data[8,:]
    trial_num = behav_data[10,:]-1 # set first trial to 0

    # extract task variables
    var_names = list(variables.dtype.names)
    variable_dict = {var_names[ind]:variables[var_names[ind]][0][0][0] for ind, item in enumerate(var_names)}
    floorLength = float(variable_dict['floorLength'])
    funnelLength = float(variable_dict['funnelLength'])
    floorWidth = float(variable_dict['floorWidth'])
    funnelWidth = float(variable_dict['funnelWidth'])
    wallHeight = float(variable_dict['wallHeight'])
    hideCuePast = float(variable_dict['hideCuePast'])
    fractionNoChecker = float(variable_dict['fractionNoChecker'])
    penaltyProb = float(variable_dict['penaltyProb'])
    feedbackDelay = float(variable_dict['feedbackDelay'])
    rewardDelay = float(variable_dict['rewardDelay'])
    itiCorrect = float(variable_dict['itiCorrect'])
    itiMissBase = float(variable_dict['itiMissBase'])
    
    # generate note for maze configuration
    maze_note = f'maze_stem_length={floorLength};maze_stem_width={floorWidth};maze_arm_length={funnelLength};maze_arm_width={funnelWidth};cue_delay_length={delay};wall_height={wallHeight};max_position={hideCuePast};frac_non_visually_guided_trials={frac_no_checker};choice_bias_penalty={penalty};feedback_delay={feedbackDelay};reward_delay={rewardDelay};iti_correct={itiCorrect};iti_incorrect={itiMissBase}'

    # read sync file (rows: pitch, roll, yaw, imaging frame, vermin frame, lick signals)
    sync_data = h5py.File(sync_loc, 'r')['sweep_0001']['analogScans']
    sync_rate = 2000.0
    session_duration = sync_data.shape[1]/sync_rate

    # get session start time by looking at the modification time of the sync file
    m_time = os.path.getmtime(sync_loc)
    session_start_time = datetime.fromtimestamp(m_time-session_duration)

    
    
    
    #### NWB FILE INITIATION ####
    # create NWB file
    nwbfile = NWBFile(
        session_description = f'Mouse performing a dynamic navigation task with calcium imaging in {fov_center_area} layer {layer}',
        identifier = f'Mouse_{mouse_id}_session_date_{session_date}',
        session_start_time = session_start_time, #datetime.strptime(str(session_date), '%Y-%m-%d'),
        session_id = f'mouse_{mouse_id}_session_date_{session_date}_area_{fov_center_area}_L{layer_name}_{img_type_name}_imaging',      
        keywords = ['mouse', 'decision-making', 'navigation', 'virtual reality', 'cortex', 'calcium imaging', 'two-photon imaging'],
        surgery = sugery_note[mouse_id],
        virus = virus_note[mouse_id],
        stimulus_notes = maze_note,
        notes = session_note,
        experimenter = 'Tseng, Shih-Yi',   
        experiment_description = f'Mouse performing a dynamic navigation task with calcium imaging in {fov_center_area} layer {layer}',
        lab = 'Harvey Lab',                                       
        institution = 'Harvard Medical School',               
        related_publications = 'doi:10.1016/j.neuron.2022.05.012'
    )
    
    # fetch mouse info
    mouse_name, doa, training_start = (swac.Mouse & key).fetch1('name', 'doa', 'training_start')

    # add subject information to NWB file
    nwbfile.subject = Subject(
        subject_id = str(mouse_id),
        date_of_birth = datetime.combine(doa-timedelta(days=56), datetime.min.time()), # date of arrival minus 8 weeks
        description = f'Mouse {mouse_id} {mouse_name}, date of arrival {doa} (~8 weeks old; date of birth is approximated), training started on {training_start}',
        species = 'Mus musculus', 
        sex = 'M',
        strain = 'C57BL/6J',
        genotype = 'wildtype'
    )
    
    
    
    
    #### ADD SYNC DATA ####
    # create TimeSeries for signals in sync data
    vel_raw = TimeSeries(
        name = 'raw_velocity_timeseries',
        data = sync_data[0:3,:].T,
        unit = 'A.U.',
        rate = sync_rate,
        description = 'Raw velocity of the spherical treadmill sampled at 2000 Hz, array of the shape: timestampes x [pitch, roll, yaw]'
    )

    imaging_frame_raw = TimeSeries(
        name = 'imaging_frame_timeseries',
        data = sync_data[3,:],
        unit = 'A.U.',
        rate = sync_rate,
        description = 'Two photon imaging frame signals sampled at 2000 Hz'
    )

    virmen_frame_raw = TimeSeries(
        name = 'virmen_frame_timeseries',
        data = sync_data[4,:],
        unit = 'A.U.',
        rate = sync_rate,
        description = 'Virmen frame signals sampled at 2000 Hz'
    )

    lick_raw = TimeSeries(
        name = 'lick_timeseries',
        data = sync_data[5,:],
        unit = 'A.U.',
        rate = sync_rate,
        description = 'Analog lick signals sampled at 2000 Hz (some sessions have signal loss due to acquisition problems, so not a reliable signal)'
    )
    
    # add sync timeseries data to acquisition
    nwbfile.add_acquisition(vel_raw)
    nwbfile.add_acquisition(imaging_frame_raw)
    nwbfile.add_acquisition(virmen_frame_raw)
    nwbfile.add_acquisition(lick_raw)   
    
    
    
    
    #### ADD VIRMEN ACQUISITION DATA ####
    # fetch sync object
    syncObj = (swac.CaSourceExtraction & key).fetch1('sources_sync_data')
    sync_names = list(syncObj.dtype.names)
    syncDict = {sync_names[ind]:syncObj[sync_names[ind]][0][0] for ind, item in enumerate(sync_names)}
     
    # get length for behavioral data and sync data of Virmen frames 
    behav_timeseries_length_verm = behav_data.shape[1]
    behav_timeseries_length_sync = syncDict['vermMidTimes'].shape[0]

    # find the shorter one
    behav_timeseries_length = np.minimum(behav_timeseries_length_sync, behav_timeseries_length_verm)
    
    # create TimeSeries for signals in Virmen behavioral data
    virmen_timestamps = ((syncDict['vermMidTimes'][:behav_timeseries_length])/sync_rate).flatten() # we have extra virmen frames from sync data than behavioral data


    # create spatial series for frame aligned position
    virmen_position_raw = SpatialSeries(
        name = 'Virmen_forward_lateral_positions', 
        description = 'Raw forward and lateral position in Virmen frames (sampled at ~30 Hz)',
        data = np.stack((for_pos[:]/100., lat_pos/100.),axis=1)[:behav_timeseries_length,:], # 1 Virmen unit ~-= 1 cm, convert from cm to m
        timestamps = virmen_timestamps,
        unit = 'm',
        reference_frame = '(0, 0) is the start point (centered) of the maze stem'
    )

    # wrap the SpatialSeries in a Position object
    position_obj_raw = Position(
        name = 'Virmen_positions',
        spatial_series = virmen_position_raw)


    virmen_vel_raw = TimeSeries(
        name = 'Virmen_forward_lateral_velocity_timeseries',
        data = np.stack((for_vel_verm, lat_vel_verm),axis=1)[:behav_timeseries_length,:],
        unit = 'A.U.',
        timestamps = virmen_position_raw, 
        description = 'Raw forward and lateral velocity (in maze) in Virmen frames (sampled at ~30 Hz)'
    )


    trial_phase_raw = TimeSeries(
        name = 'Virmen_trial_phase_timeseries',
        data = trial_phase[:behav_timeseries_length],
        unit = 'N.A.',
        timestamps = virmen_position_raw, 
        description = 'Trial phase in Virmen frames (sampled at ~30 Hz), 0: maze, -1: feedback delay/feedback, -2: visual feedback onset, 1: ITI'
    )

    trial_num_raw = TimeSeries(
        name = 'Virmen_trial_number_timeseries',
        data = trial_num[:behav_timeseries_length],
        unit = 'N.A.',
        timestamps = virmen_position_raw, 
        description = 'Trial number in Virmen frames (sampled at ~30 Hz)'
    )

    reward_raw = TimeSeries(
        name = 'Virmen_reward_timeseries',
        data = reward[:behav_timeseries_length].astype(bool),
        unit = 'N.A.',
        timestamps = virmen_position_raw, 
        description = 'Reward in Virmen frames (sampled at ~30 Hz), 1: reward delivered'
    )

    world_raw = TimeSeries(
        name = 'Virmen_maze_world_timeseries',
        data = world[:behav_timeseries_length],
        unit = 'N.A.',
        timestamps = virmen_position_raw, 
        description = 'Maze world in Virmen frames (sampled at ~30 Hz), 1:WLV, 2:BRV, 3:WRV, 4:BLV, 5:WLN, 6:BRN, 7:WRN, 8:BLN; B/W:black/white cue, L/R:left/right reward location, V/N:visually-guided/normal trial'
    ) 
    
    # add Virmen timeseries data to acquisition
    nwbfile.add_acquisition(position_obj_raw)
    nwbfile.add_acquisition(virmen_vel_raw)
    nwbfile.add_acquisition(trial_num_raw)
    nwbfile.add_acquisition(trial_phase_raw)
    nwbfile.add_acquisition(reward_raw)
    nwbfile.add_acquisition(world_raw)
    
    #### ADD TRIAL DATA ####
    # fetch framed aligned behavioral data
    tI = (swac_beh.BehaviorStructure & key).fetch1('behav_struct')
    trialVar = tI.dtype.names
    trial_dict = {trialVar[ind]:tI[trialVar[ind]][0][0] for ind, item in enumerate(trialVar)}
    nTrials = trial_dict['trialCueB'].shape[1]
    
    # fetch strategy variables (order for strat_pred: WR, WL, BR, BL)
    h, mi, rule_belief, strat_preds = (swac_beh.BehaviorRNN & key).fetch1('strat_h','strat_mi','strat_rule','strat_preds')

    # compute derived strategy variables
    mpa, mpa_cho, mpa_cue = model_prob_of_action(strat_preds, trial_dict)
    signed_bias = strat_preds[[1,3],:].sum(axis=0) - strat_preds[[0,2],:].sum(axis=0)
    association_matrix = strat_preds*2 # multiply by two to obtain conditional probability values
    
    # find rule (context) for every trial
    rule_A_world = [3,4,7,8] # BL,WR
    rule_B_world = [1,2,5,6] # WL,BR
    is_ruleA = np.array([True if this_world in rule_A_world else False for this_world in syncDict['tWorld']])
    is_switch = np.array([True if this_trial in trial_dict['switches'] else False for this_trial in range(nTrials)])
    
    # add and define all the trial column
    nwbfile.add_trial_column(name='is_vis', description='whether the trial was a visually guided trial (True: visually guided, False: non-visually guided)')
    nwbfile.add_trial_column(name='is_ruleA', description='whether the trial happened during rule A (True: rule A (BL,WR), False: rule B(WL,BR))')
    nwbfile.add_trial_column(name='is_switch', description='whether a rule switch happened on the trial (True: switch, False: non-switch)')

    nwbfile.add_trial_column(name='is_cueB', description='whether the trial had a black cue (True: black, False: white)')
    nwbfile.add_trial_column(name='is_choL', description='whether the mouse made a left choice on the trial (True: left, False: right)')
    nwbfile.add_trial_column(name='is_correct', description='whether the trial was correct (True: correct, False: incorrect)')

    # nwbfile.add_trial_column(name='prevCueB', description='whether the previous trial had a black cue (1: black, 0: white, or NaN for trial 0)')
    # nwbfile.add_trial_column(name='prevChoL', description='whether the mouse made a left choice on the previous trial (1: left, 0: right, or NaN for trial 0)')
    # nwbfile.add_trial_column(name='prevCorr', description='whether the previous trial was correct (1: correct, 0: incorrect, or NaN for trial 0)')

    nwbfile.add_trial_column(name='association_mat', description='behavioral LSTM predicted conditional probability, order: P(R|W), P(L|W), P(R|B), P(L|B)')
    nwbfile.add_trial_column(name='rule_belief', description='the rule belief on the trial (pos: rule B, neg: rule A)')
    nwbfile.add_trial_column(name='signed_bias', description='the singed choice bias on the trial (pos: left bias, neg: right bias)')
    nwbfile.add_trial_column(name='rule_following', description='the rule-following on the trial (between -0.5 ~ 0.5)')
    nwbfile.add_trial_column(name='bias_following', description='the bias-following on the trial (between -0.5 ~ 0.5)')
    nwbfile.add_trial_column(name='prob_actual_cho', description='the probability of actual choice on the trial, i.e. P(actual choice|actual cue)')

    nwbfile.add_trial_column(name='trial_onset_plane_frame_idx', description='correspoding imaging frame index for this trial onset for each imaging plane')
    nwbfile.add_trial_column(name='trial_offset_plane_frame_idx', description='correspoding imaging frame index for this trial offset for each imaging plane')
    
    # loop over trials and add data into trial table
    for i_trial in range(nTrials):
        # find vermin frame for trial onset and offset
        start_time_verm_frame = int(syncDict['trialStarts'][i_trial][0])
        if i_trial != nTrials-1:
            stop_time_verm_frame = int(syncDict['trialStarts'][i_trial+1][0])-1
        else: 
            stop_time_verm_frame = virmen_timestamps.shape[0]-1 # choose last virmen frame as trial offset for the last trial

        # add individual trials
        nwbfile.add_trial(
            start_time = syncDict['vermMidTimes'][start_time_verm_frame][0]/sync_rate,
            stop_time = syncDict['vermMidTimes'][stop_time_verm_frame][0]/sync_rate,
            is_vis = bool(trial_dict['trialVis'].flatten()[i_trial]),
            is_ruleA = is_ruleA[i_trial],
            is_switch = is_switch[i_trial],
            is_cueB = bool(trial_dict['trialCueB'].flatten()[i_trial]),
            is_choL = bool(trial_dict['trialChoL'].flatten()[i_trial]),
            is_correct = bool(trial_dict['trialRew'].flatten()[i_trial]),
    #         prevCueB = trial_dict['prevTrialCueB'].flatten()[i_trial],
    #         prevChoL = trial_dict['prevTrialChoL'].flatten()[i_trial],
    #         prevCorr = trial_dict['prevTrialRew'].flatten()[i_trial],
            association_mat = association_matrix[:,i_trial],
            rule_belief = rule_belief.flatten()[i_trial],
            signed_bias = signed_bias[i_trial],
            rule_following = mpa_cue[i_trial],
            bias_following = mpa_cho[i_trial]-0.5,
            prob_actual_cho = mpa[i_trial],
            trial_onset_plane_frame_idx = trial_dict['trialOnsets'][:,i_trial].astype(int),
            trial_offset_plane_frame_idx = trial_dict['trialOffsets'][:,i_trial].astype(int)
        )
        
        
        
        
    #### ADD EPOCH DATA ####
    # add epoch columns
    nwbfile.add_epoch_column(name = 'epoch_type', description = 'type of epoch: maze, maze_stem, maze_arm, feedback_delay, feedback, iti, feedback_and_iti')
    nwbfile.add_epoch_column(name = 'trial_id', description = 'trial number for this epoch')
    
    # loop over trials and add data into epoch table
    for i_trial in range(nTrials):
        # identify virmen frames for each epoch
        trial_verm_frame_idx = trial_num == i_trial
        maze_onset = np.where(np.logical_and(trial_phase == 0, trial_verm_frame_idx))[0][0]
        stem_onset = maze_onset.copy()

        # find all frames in feedback delay and feedback period
        all_feedback_frames = np.where(np.logical_and(trial_phase == -1, trial_verm_frame_idx))[0]

        # if feedback frames exist
        if all_feedback_frames.shape[0]>0:
            feedback_delay_exist = True
            feedback_delay_onset = all_feedback_frames[0]
            feedback_iti_onset = feedback_delay_onset.copy()
            maze_offset = feedback_delay_onset-1
            funnel_offset = maze_offset.copy()
        else:    
            feedback_delay_exist = False
            maze_offset = np.where(np.logical_and(trial_phase == 0, trial_verm_frame_idx))[0][-1]

        # find all positions in the maze
        these_pos = for_pos[maze_onset:maze_offset+1]
        if these_pos[-1] > floorLength:
            funnel_exist = True
            stem_offset = np.where(np.logical_and(for_pos <= floorLength, trial_verm_frame_idx))[0][-1]
            funnel_onset = stem_offset + 1
        else:
            funnel_exist = False
            stem_offset = maze_offset.copy()

        if feedback_delay_exist: # if feedback delay/feedback exist
            feedback_onset = np.where(np.logical_and(trial_phase == -2, trial_verm_frame_idx))[0]
            feedback_offset = np.where(trial_num == i_trial)[0][-1]

            # if feedback onset frame doesn't exist
            if feedback_onset.shape[0] == 0:
                feedback_exist = False
                feedback_delay_offset = all_feedback_frames[-1] # feedback delay offset is the last frame in all_feedback_frames
                feedback_iti_offset = feedback_delay_offset.copy()
            elif feedback_onset.shape[0] > 0:
                feedback_exist = True
                feedback_delay_offset = feedback_onset[0]-1

                if i_trial < nTrials-1: # last trial does not have ITI
                    iti_frames = np.where(np.logical_and(trial_phase == 1,trial_num == (i_trial + 1)))[0] # this trial's ITI is labeled as next trial in trial_num
                    feedback_iti_offset = iti_frames[-1]
                else:
                    feedback_iti_offset = feedback_offset.copy()

        # add epochs    
        nwbfile.add_epoch(
            start_time = syncDict['vermMidTimes'].flatten()[maze_onset]/sync_rate,
            stop_time = syncDict['vermMidTimes'].flatten()[maze_offset]/sync_rate,
            trial_id = i_trial,
            epoch_type = 'maze'
        )

        nwbfile.add_epoch(
            start_time = syncDict['vermMidTimes'].flatten()[stem_onset]/sync_rate,
            stop_time = syncDict['vermMidTimes'].flatten()[stem_offset]/sync_rate,
            trial_id = i_trial,
            epoch_type = 'maze_stem'
        )

        if funnel_exist:
            nwbfile.add_epoch(
                start_time = syncDict['vermMidTimes'].flatten()[funnel_onset]/sync_rate,
                stop_time = syncDict['vermMidTimes'].flatten()[funnel_offset]/sync_rate,
                trial_id = i_trial,
                epoch_type = 'maze_arm'
            )

        if feedback_delay_exist:
            nwbfile.add_epoch(
                start_time = syncDict['vermMidTimes'].flatten()[feedback_delay_onset]/sync_rate,
                stop_time = syncDict['vermMidTimes'].flatten()[feedback_delay_offset]/sync_rate,
                trial_id = i_trial,
                epoch_type = 'feedback_delay'
            )
            if feedback_exist:
                nwbfile.add_epoch(
                    start_time = syncDict['vermMidTimes'].flatten()[feedback_onset[0]]/sync_rate,
                    stop_time = syncDict['vermMidTimes'].flatten()[all_feedback_frames[-1]]/sync_rate,
                    trial_id = i_trial,
                    epoch_type = 'feedback'
                )


        if i_trial < nTrials-1: # last trial does not have ITI
            nwbfile.add_epoch(
                start_time = syncDict['vermMidTimes'].flatten()[iti_frames[0]]/sync_rate,
                stop_time = syncDict['vermMidTimes'].flatten()[iti_frames[-1]]/sync_rate,
                trial_id = i_trial,
                epoch_type = 'iti'
            )

        if feedback_delay_exist:
            nwbfile.add_epoch(
                start_time = syncDict['vermMidTimes'].flatten()[feedback_iti_onset]/sync_rate,
                stop_time = syncDict['vermMidTimes'].flatten()[feedback_iti_offset]/sync_rate,
                trial_id = i_trial,
                epoch_type = 'feedback_and_iti'
            )
            
    
    
    
    #### ADD PROCESSED BEHAVIORAL DATA ####
    # create processing module for processed behavioral data
    behavior_module = nwbfile.create_processing_module(
        name = 'behavior', 
        description = 'processed behavioral data'
    )

    # reshape frameMidTimes
    if img_type == 'volume':
        nPlanes = 5
        nImgPlanes = 4
        frameMidTimes_reshape = syncDict['frameMidTimes'].reshape(-1,nPlanes)
    elif img_type == 'plane':
        nPlanes = 1
        nImgPlanes = 1
        # create 6 Hz frames (velRNN is modeled at 6 Hz) 
        nFrames_vol = int(np.floor(syncDict['frameMidTimes'].shape[0]/5))
        frameMidTimes_reshape = syncDict['frameMidTimes'][:nFrames_vol*5,:].reshape(-1,5)

    # fetch frame aligned velocity
    frame_pitch, frame_roll, frame_yaw = (swac.FrameAlignedVel & key).fetch1('frame_pitch', 'frame_roll', 'frame_yaw')

    # fetch velRNN values
    valid_frames_rnn, cho_f_score_rnn, cho_r_score_rnn, cue_f_score_rnn, cue_r_score_rnn = (swac_beh.VelPredRNN & key).fetch1(
        'valid_frames_rnn','cho_f_score_rnn','cho_r_score_rnn','cue_f_score_rnn','cue_r_score_rnn')

    valid_frames_rnn = valid_frames_rnn.reshape(-1,)

    cho_pred_f = np.full(valid_frames_rnn.shape, np.NaN)
    cho_pred_r = np.full(valid_frames_rnn.shape, np.NaN)
    cue_pred_f = np.full(valid_frames_rnn.shape, np.NaN)
    cue_pred_r = np.full(valid_frames_rnn.shape, np.NaN)

    cho_pred_f[valid_frames_rnn] = cho_f_score_rnn.reshape(-1,)
    cho_pred_r[valid_frames_rnn] = cho_r_score_rnn.reshape(-1,)
    cue_pred_f[valid_frames_rnn] = cue_f_score_rnn.reshape(-1,)
    cue_pred_r[valid_frames_rnn] = cue_r_score_rnn.reshape(-1,)
    
    # unfold framed aligned signals
    posL_unfold = trial_dict['posL'].T.reshape(-1,)
    posF_unfold = trial_dict['posF'].T.reshape(-1,)
    frameTrialMem_unfold = trial_dict['frameTrialMem'].T.reshape(-1,) - 1 # convert first trial to 0
    choFrameOffsets_unfold = trial_dict['choFrameOffsets'].T.reshape(-1,)
    pitch_unfold = frame_pitch.T.reshape(-1,)
    roll_unfold = frame_roll.T.reshape(-1,)
    yaw_unfold = frame_yaw.T.reshape(-1,)
    
    # create spatial series for frame aligned position
    position_spatial_series = SpatialSeries(
        name = 'frame_aligned_forward_and_lateral_position', 
        description = 'forward and lateral position aligned to imaging frames (30 Hz)',
        data = np.stack((posF_unfold/100., posL_unfold/100.), axis = 1), # convert from cm to m 
        unit = 'm',
        timestamps = syncDict['frameMidTimes'].flatten()/sync_rate,
        reference_frame = '(0, 0) is the start point (centered) of the maze stem'
    )

    # wrap the SpatialSeries in a Position object
    position_obj = Position(
        name = 'frame_aligned_position',
        spatial_series = position_spatial_series)

    # add Position into behavioral processing module
    behavior_module.add(position_obj)
    
    # create time series for frame aligned velocity
    velocity_time_series = TimeSeries(
        name = 'frame_aligned_pitch_roll_yaw_velocity',
        data = np.stack((pitch_unfold, roll_unfold, yaw_unfold), axis = 1), 
        timestamps = position_spatial_series, # reusing the timestamps (frameMidTimes) stored in position_spatial_series
        description = 'velocity of the spherical treadmill (pitch, roll, yaw) aligned to imaging frames (30 Hz)',
        unit = 'A.U.'
    )

    # wrap the TimeSeries in a BehavioralTimeSeries object
    behavioral_time_series_vel = BehavioralTimeSeries(
        time_series = velocity_time_series,
        name = 'frame_aligned_velocity'
    )

    # add the BehavioralTimeSeries into behavioral processing module
    behavior_module.add(behavioral_time_series_vel)
    
    # create time series for velRNN
    rnn_time_series = TimeSeries(
        name = 'velocity_RNN_prediction_for_choice_and_cue',
        data = np.stack((cho_pred_f, cho_pred_r, cue_pred_f, cue_pred_r),axis=1), 
        timestamps = frameMidTimes_reshape[:,2]/sync_rate, # take the mid slice 
        unit = 'N.A.',
        description = 'velocity RNN prediction (forward choice, reverse choice, forward cue, reverse cue) aligned to mid imaging frames (idx=2) of each 6 Hz volume (or pseudovolume for plane imaging)',
    )

    # wrap the TimeSeries in a BehavioralTimeSeries object
    behavioral_time_series_rnn = BehavioralTimeSeries(
        time_series = rnn_time_series,
        name = 'velocity_RNN_prediction_for_choice_and_cue'
    )

    # add the BehavioralTimeSeries into behavioral processing module
    behavior_module.add(behavioral_time_series_rnn)
    
    # create time series for frameTrialMem
    frameTrialMem_time_series = TimeSeries(
        name = 'frame_aligned_trial_number',
        data = frameTrialMem_unfold, 
        timestamps = position_spatial_series, # reusing the timestamps (frameMidTimes) stored in position_spatial_series
        unit = 'N.A.',
        description = 'trial number aligned to imaging frames',
    )


    # create time series for choFrameOffsets
    choFrameOffsets_time_series = TimeSeries(
        name = 'frame_aligned_time_from_choice_point',
        data = choFrameOffsets_unfold, 
        timestamps = position_spatial_series,
        unit = 's',
        description = 'time elapsed (sec) from choice point aligned to imaging frames (neg: maze, pos: feedback and ITI)',
    )

    # create time series for ver2frame
    verm2frame_time_series = TimeSeries(
        name = 'verm_to_frame_index_conversion',
        data = syncDict['verm2frame'].flatten()-1, # minus one to convert from matlab index to python index
        timestamps = syncDict['vermMidTimes'].flatten()/sync_rate,
        unit = 'N.A.',
        description = 'correspoding imaging frame index for each verm frame'
    )

    # create time series for frame2verm
    frame2verm_time_series = TimeSeries(
        name = 'frame_to_verm_index_conversion',
        data = syncDict['frame2verm'].flatten()-1,  # minus one to convert from matlab index to python index
        timestamps = position_spatial_series, 
        unit = 'N.A.',
        description = 'correspoding verm frame index for each imaging frame'
    )

    # add these TimeSeries into behavioral processing module
    behavior_module.add(frameTrialMem_time_series)
    behavior_module.add(choFrameOffsets_time_series)
    behavior_module.add(verm2frame_time_series)
    behavior_module.add(frame2verm_time_series)

    # create plane number for imaging frames
    if img_type == 'volume':
        nFrames_per_plane = cho_pred_f.shape[0]
        plane_idx = repmat(np.array([0,1,2,3,4]), 1, nFrames_per_plane).reshape(-1,)

        # create time series for plane_num
        plane_idx_time_series = TimeSeries(
            name = 'plane_idx_for_imaging_frames',
            data = plane_idx, 
            timestamps = position_spatial_series,
            unit = 'N.A.',
            description = 'plane index for imaging frames'
        )

        # add the TimeSeries into behavioral processing module
        behavior_module.add(plane_idx_time_series)
        
        
        
    #### ADD 2P IMAGING DATA ####
    # create processing modules for 2p imaging 
    ophys_module = nwbfile.create_processing_module(
        name = 'ophys', 
        description = 'two photon calcium imaging processed data'
    )

    # create ImageSegmentation object
    img_seg = ImageSegmentation()
    
    # fetch FOV center location in CCF
    tmat, center_ml_ccf_mm, center_ap_ccf_mm, depth_mm = (swac.FOVtoCCF & key).fetch1('tmat_fov_to_ccf_mm','center_ml_ccf_mm', 'center_ap_ccf_mm', 'depth_mm')

    # compute pixel size and orgin location
    fov_ap_size = 675./1000
    fov_ml_size = 750./1000
    mm_per_pixel_ap = fov_ap_size/512
    mm_per_pixel_ml = fov_ml_size/512
    # FOV direction: x-axis: posterior to anterior; y-axis: laterl (left) to medial (right)
    origin_x = np.matmul(tmat, np.array([[1,1,1]]).T).reshape(-1,)[1] # ap # the origin is [1,1,1] because it was the first pixel in Matlab indexing
    origin_y = np.matmul(tmat, np.array([[1,1,1]]).T).reshape(-1,)[0] # ml

    # create Device, OptionalChannel and ImagingPlane
    device = nwbfile.create_device(
        name = 'two_photon_microscope', 
        description = 'Two-photon microscope',
        manufacturer = 'Harvey Lab custom-bulit two-photon microscope (Odin)'
    )

    # create OpticalChannel for GCaMP
    optical_channel = OpticalChannel(
        name = 'OpticalChannel_GCaMP', 
        description = 'optical channel for GCaMP', 
        emission_lambda = 525.
    )

    # create ImagingPlane for GCaMP
    imaging_plane = nwbfile.create_imaging_plane(
        name = 'ImagingPlane_GCaMP6s_920nm',
        optical_channel = optical_channel,
        imaging_rate = 30.,
        description = f'{img_type} calcium imaging in {fov_center_area} layer {layer}',
        device = device,
        excitation_lambda = 920.,
        indicator = 'GCaMP6s',
        location = fov_center_area,
        grid_spacing = [mm_per_pixel_ap, mm_per_pixel_ml, step_size/1000.],
        grid_spacing_unit = 'mm',
        origin_coords = [origin_x, origin_y, depth_mm],
        origin_coords_unit = 'mm',
        reference_frame = f'[p->a, l->m, depth from pia] in minimeters from bregma in Allen Institute Mouse Common Coordinate Framework (CCF); i.e. directionality of the imaging FOV: x-axis: posterior to anterior; y-axis: laterl (left) to medial (right); however, the FOV axes does not run parallelly to the CCF coordinate system. The following transformation matrix can be applied to a column vector of pixel coordiate ([x,y,1].T) to [ml, ap, _] in mm in CCF: np.array([[{tmat[0,0]:.6f},{tmat[0,1]:.6f},{tmat[0,2]:.6f}],[{tmat[1,0]:.6f},{tmat[1,1]:.6f},{tmat[1,2]:.6f}],[{tmat[2,0]:.6f},{tmat[2,1]:.6f},{tmat[2,2]:.6f}]])'
    )

    # create OpticalChannel for mTagBFP2
    optical_channel_mTagBFP2 = OpticalChannel(
        name = 'OpticalChannel_mTagBFP2', 
        description = 'optical channel for mTagBFP2', 
        emission_lambda = 450.
    )

    # create ImagingPlane for mTagBFP2
    imaging_plane_mTagBFP2 = nwbfile.create_imaging_plane(
        name = 'ImagingPlane_mTagBFP2_850nm',
        optical_channel = [optical_channel_mTagBFP2, optical_channel],
        imaging_rate = 30.,
        description = 'mTagBFP2/GCaMP6s imaging for AAVretro labeling',
        device = device,
        excitation_lambda = 850.,
        indicator = 'mTagBFP2',
        location = fov_center_area,
        grid_spacing = [mm_per_pixel_ap, mm_per_pixel_ml, step_size/1000.],
        grid_spacing_unit = 'mm',
        origin_coords = [origin_x, origin_y, depth_mm],
        origin_coords_unit = 'mm',
        reference_frame = f'[p->a, l->m, depth from pia] in minimeters from bregma in Allen Institute Mouse Common Coordinate Framework (CCF); i.e. directionality of the imaging FOV: x-axis: posterior to anterior; y-axis: laterl (left) to medial (right); however, the FOV axes does not run parallelly to the CCF coordinate system. The following transformation matrix can be applied to a column vector of pixel coordiate ([x,y,1].T) to [ml, ap, _] in mm in CCF: np.array([[{tmat[0,0]:.6f},{tmat[0,1]:.6f},{tmat[0,2]:.6f}],[{tmat[1,0]:.6f},{tmat[1,1]:.6f},{tmat[1,2]:.6f}],[{tmat[2,0]:.6f},{tmat[2,1]:.6f},{tmat[2,2]:.6f}]])'
    )

    # create OpticalChannel for mScarlet
    optical_channel_mScarlet = OpticalChannel(
        name = 'OpticalChannel_mScarlet', 
        description = 'optical channel for mScarlet', 
        emission_lambda = 625.
    )

    # create ImagingPlane for mScarlet
    imaging_plane_mScarlet = nwbfile.create_imaging_plane(
        name = 'ImagingPlane_mScarlet_800nm',
        optical_channel = [optical_channel_mScarlet, optical_channel],
        imaging_rate = 30.,
        description = 'mScarlet/GCaMP6s imaging for AAVretro labeling',
        device = device,
        excitation_lambda = 800.,
        indicator = 'mScarlet',
        location = fov_center_area,
        grid_spacing = [mm_per_pixel_ap, mm_per_pixel_ml, step_size/1000.],
        grid_spacing_unit = 'mm',
        origin_coords = [origin_x, origin_y, depth_mm],
        origin_coords_unit = 'mm',
        reference_frame = f'[p->a, l->m, depth from pia] in minimeters from bregma in Allen Institute Mouse Common Coordinate Framework (CCF); i.e. directionality of the imaging FOV: x-axis: posterior to anterior; y-axis: laterl (left) to medial (right); however, the FOV axes does not run parallelly to the CCF coordinate system. The following transformation matrix can be applied to a column vector of pixel coordiate ([x,y,1].T) to [ml, ap, _] in mm in CCF: np.array([[{tmat[0,0]:.6f},{tmat[0,1]:.6f},{tmat[0,2]:.6f}],[{tmat[1,0]:.6f},{tmat[1,1]:.6f},{tmat[1,2]:.6f}],[{tmat[2,0]:.6f},{tmat[2,1]:.6f},{tmat[2,2]:.6f}]])'
    )
    
    # fetch vessel pattern image
    vessel_img = (swac.FOVinitialization & key).fetch1('overview')
    
    if vessel_img is not None:
        # create objects for vessel pattern image
        vessel = GrayscaleImage(
            name = 'vessel_img',
            data = vessel_img,
            description = 'image of vessel pattern at brain surface for this FOV'
        )
        # put image in an Images container
        vessel_image = Images(
            name = 'vessel_img',
            images = [vessel],
            description = 'image of vessel pattern at brain surface for this FOV',
        )
        # add to ophys module
        ophys_module.add(vessel_image)
        
    else: print('Vessel image does not exist!')
        
    # fetch gcamp and retrograde images 
    isRetrolabel = len(swac.RetrolabelUnmixing & key) > 0
    if isRetrolabel:
        blue_img, red_img, green850_img, green800_img = (swac.RetrolabelUnmixing & key).fetch1('blue', 'red', 'green850', 'green800')
        if img_type == 'plane':
            blue_img = blue_img[:,:,np.newaxis]
            red_img = red_img[:,:,np.newaxis]
            green850_img = green850_img[:,:,np.newaxis]
            green800_img = green800_img[:,:,np.newaxis]

        for nPlane in range(nImgPlanes):

            # create objects for gcamp and retrograde label images
            gcamp_img_850 = GrayscaleImage(
                name = 'GCaMP_850nm',
                data = green850_img[:,:,nPlane],
                description = 'static unmixed GCaMP image of the FOV with excitation 850 nm'
            )
            gcamp_img_800 = GrayscaleImage(
                name = 'GCaMP_800nm',
                data = green800_img[:,:,nPlane],
                description = 'static unmixed GCaMP image of the FOV with excitation 800 nm'
            )
            mTagBFP2_img = GrayscaleImage(
                name = 'mTagBFP2_850nm',
                data = blue_img[:,:,nPlane],
                description = 'static unmixed mTagBFP2 image of the FOV with excitation 850 nm'
            )
            mScarlet_img = GrayscaleImage(
                name = 'mScarlet_800nm',
                data = red_img[:,:,nPlane],
                description = 'static unmixed mScarlet image of the FOV with excitation 800 nm'
            )

            # put image in an Images container
            static_images = Images(
                name = f'static_GCaMP_and_retrograde_label_image_plane_{nPlane}',
                images = [gcamp_img_850, gcamp_img_800, mTagBFP2_img, mScarlet_img],
                description = "static unmixed images for GCaMP and retrograde labels of the FOV",
            )

            ophys_module.add(static_images)
            
    else: print('Retrolabel images does not exist!')
            
            
    # fetch retrolabel threshold
    thresh_l23b, thresh_l23r, thresh_l5r = (swac.RetrolabelThresh & 'thresh_type="M"').fetch1('thresh_l23b', 'thresh_l23r', 'thresh_l5r')

    # create area-number mapping
    area_name_dict = {0:'V1',1:'PM',2:'AM',3:'MM',4:'RSC',5:'A'}

    # loop over imaging planes
    for nPlane in range(nImgPlanes):

        # create PlaneSegmentation for this slice
        ps = img_seg.create_plane_segmentation(
            name = 'PlaneSegmentation_' + str(nPlane),
            description = f'Plane segmentation of imagine plane {nPlane}',
            imaging_plane = imaging_plane  
        )

        # add extra fields for source location and retrograde labeling
        ps.add_column(name = 'ml', description = 'ml location (mm from bregma)')
        ps.add_column(name = 'ap', description = 'ap location (mm from bregma)')
        ps.add_column(name = 'depth', description = 'depth (mm from pia)')
        ps.add_column(name = 'area', description = 'cortical area where this cell is located')

        # add column for retrolabel
        if isRetrolabel:
            ps.add_column(name = 'mTagBFP2', description = 'whether this cell is labeled with mTagBFP2')
            ps.add_column(name = 'mScarlet', description = 'whether this cell is labeled with Scarlet')

        # fetch source info and activity
        source_ind, source_filter, source_ml, source_ap, source_depth, source_df, source_deconv, source_g, source_lam = (swac.SourceLocation*swac.SourceTrace*swac.Source & key & 'source_type="cell"'& f'source_slice={nPlane+1}').fetch(
            'source_ind', 'source_filter', 'source_ml', 'source_ap', 'source_depth', 'source_df', 'source_deconv', 'source_g', 'source_lam', order_by='source_ind')

        # fetch retrograde label
        if isRetrolabel:
            bscore, rscore = (swac.Retrolabel*swac.Source & key & 'source_type="cell"'& f'source_slice={nPlane+1}').fetch('bscore', 'rscore', order_by='source_ind')
            if layer == '2/3':
                is_blue = bscore >= thresh_l23b
                is_red = rscore >= thresh_l23r
            elif layer == '5':
                is_red = rscore >= thresh_l5r
                is_blue = np.zeros_like(is_red, dtype=bool) # for layer 5, put all blue to False

        # classify source into area
        source_area_ind = partition_areas(source_ml, source_ap)
        source_area = source_area_ind['PM']+2*source_area_ind['AM']+3*source_area_ind['MM']+4*source_area_ind['RSC']+5*source_area_ind['A/RL']
        source_area_name = [area_name_dict[this_area] for this_area in source_area]

        # add roi into PlaneSegmentation
        n_sources = source_ind.shape[0]
        for i_source in range(n_sources):
            if isRetrolabel:
                ps.add_roi(id = i_source, pixel_mask = source_filter[i_source], 
                           ml = source_ml[i_source], ap = source_ap[i_source], depth = source_depth[i_source], 
                           area = source_area_name[i_source], mTagBFP2 = is_blue[i_source], mScarlet = is_red[i_source])
            else:
                ps.add_roi(id = i_source, pixel_mask = source_filter[i_source], 
                           ml = source_ml[i_source], ap = source_ap[i_source], depth = source_depth[i_source], 
                           area = source_area_name[i_source])


        # create ROI table region 
        rt_region = ps.create_roi_table_region(
            region = [i_source for i_source in range(n_sources)],
            description = f'the ROIs in PlaneSegmentation_{nPlane}'
        )

        if img_type == 'plane':
            imaging_timestamps = syncDict['frameMidTimes'].flatten()
        elif img_type == 'volume':
            imaging_timestamps = frameMidTimes_reshape[:,nPlane]/sync_rate

        # create RoiResponseSeries for deconvolved activity and df
        deconv_activity = RoiResponseSeries(
            name = f'deconvolved_activity_plane_{nPlane}',
            description = f'Deconvolved activity for ROIs in PlaneSegmentation_{nPlane}',
            data = np.concatenate(source_deconv, axis=0).T, 
            rois = rt_region,
            unit = 'A.U.',
            timestamps = imaging_timestamps,
        )

        df = RoiResponseSeries(
            name = f'dF_over_F_plane_{nPlane}',
            description = f'dF/F for ROIs in PlaneSegmentation_{nPlane}',
            data = np.concatenate(source_df, axis=0).T, 
            rois = rt_region,
            unit = 'A.U.',
            timestamps = deconv_activity, # reusing the timestamps stored in deconv_activity
        )

        # put RoiResponseSeries for df into DfOverF
        df_over_f = DfOverF(name = f'df_over_f_plane_{nPlane}', roi_response_series = df)

        ophys_module.add(df_over_f)
        ophys_module.add(deconv_activity)

    # add ImageSegmentation to the ophys module
    ophys_module.add(img_seg)
    
    
    #### ADD LABMETADATA ####
    if is_ruleA[0]:
        initial_rule = 'A'
    else:
        initial_rule = 'B'

    if img_type == 'volume':
        imaging_type = 'multi-plane'
    else:
        imaging_type = 'single-plane'
        
    lab_meta_data = LabMetaDataSession(
        TaskParam__maze_stem_length=floorLength,
        TaskParam__maze_stem_width=floorWidth,
        TaskParam__maze_arm_length=funnelLength,
        TaskParam__maze_arm_width=funnelWidth,
        TaskParam__cue_delay_length=float(delay),
        TaskParam__wall_height=wallHeight,
        TaskParam__max_position=hideCuePast,
        TaskParam__meter_per_virmen_unit=0.01,
        TaskParam__frac_non_visually_guided_trials=frac_no_checker,
        TaskParam__choice_bias_penalty=float(penalty),
        TaskParam__feedback_delay_sec=feedbackDelay,
        TaskParam__reward_delay_sec=rewardDelay,
        TaskParam__iti_correct_sec=itiCorrect,
        TaskParam__iti_incorrect_sec=itiMissBase,
        TaskParam__switches=trial_dict['switches'].flatten().astype(int),
        TaskParam__initial_rule=initial_rule,
        AAVretroInjSite__mTagBFP2=retro_site[mouse_id]['mTagBFP2'],
        AAVretroInjSite__mScarlet=retro_site[mouse_id]['mScarlet'],
        Imaging__imaging_type=imaging_type,
        Imaging__num_of_frame_per_volume=nPlanes,
        Imaging__num_of_imaging_plane=nImgPlanes,
        Imaging__cortical_layer=layer,
        Registration__fov_center_area=fov_center_area,
        Registration__fov_center_ml_ccf_mm=center_ml_ccf_mm,
        Registration__fov_center_ap_ccf_mm=center_ap_ccf_mm, 
        Registration__fov_depth_mm=depth_mm,
        Registration__fov_plane_step_size_mm=step_size/1000., # convert from um to mm
        Registration__fov_to_ccf_transformation_matrix=tmat
    )
    nwbfile.add_lab_meta_data(lab_meta_data=lab_meta_data)
    
    
    #### SAVE NWB FILE ####
    filePath = 'Z:\\HarveyLab\\Tier1\\Shih_Yi\\NWBfile\\'+ f'{mouse_id}\\mouse_{mouse_id}_{fov_center_area}_L{layer_name}_{img_type}_imaging_{session_date}.nwb'
    with NWBHDF5IO(filePath, 'w') as io:
        io.write(nwbfile)